# Running DeepSeek R1 locally

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

c:\Users\angel\Documents\projects\ai_stuff\local_ai_agents\my_little_local_agents\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
#@title Connecting to LM Studio
import requests

# Define LM Studio endpoint
LM_STUDIO_URL = "http://127.0.0.1:1234"

# Test that LM Studio is working
try:
    response = requests.get(f"{LM_STUDIO_URL}/v1/models")
    if response.status_code == 200:
        print("✅ LM Studio is running!")
        print("Available models:", response.json())
    else:
        print("⚠️ Unable to reach LM Studio. Make sure it's running.")
except requests.exceptions.ConnectionError:
    print("❌ LM Studio is not running. Start it and try again.")

✅ LM Studio is running!
Available models: {'data': [{'id': 'deepseek-r1-distill-qwen-14b', 'object': 'model', 'owned_by': 'organization_owner'}, {'id': 'text-embedding-nomic-embed-text-v1.5', 'object': 'model', 'owned_by': 'organization_owner'}, {'id': 'deepseek-r1-distill-llama-8b', 'object': 'model', 'owned_by': 'organization_owner'}, {'id': 'deepseek-r1-distill-qwen-7b', 'object': 'model', 'owned_by': 'organization_owner'}], 'object': 'list'}


In [28]:
#title Testing out a prompt
prompt = "What is the the best fruit? Choose one."

llm_model = "deepseek-r1-distill-qwen-14b"
max_tokens = 4096
temperature = 0.7
# Define the request payload for LM Studio
payload = {
    "model": llm_model,
    "promt": prompt,
    "messages": [
        {"role": "user", "content": prompt}
        ],
    "max_tokens": max_tokens,
    "temperature": temperature
}

# Send the request to LM Studio
response = requests.post(f"{LM_STUDIO_URL}/v1/chat/completions", json=payload)

# Parse and print the response

if response.status_code == 200:
    result = response.json()
    print("🤖 LM Studio:", result["choices"][0]["message"]["content"])  # Fixed key
else:
    print("❌ Something went wrong:", response.text)

🤖 LM Studio: <think>
Okay, so I need to figure out what the best fruit is and choose one. Hmm, where do I even start? There are so many fruits out there—apples, bananas, oranges, berries, melons, citrus fruits, tropical ones like mangoes and pineapples... It's a bit overwhelming.

First, maybe I should think about what makes a fruit "good." Is it taste, nutrition, availability, versatility in recipes, or perhaps something else? Each of these factors could influence someone's choice. For example, if we're talking about taste alone, some people might say strawberries are the best because they're sweet and juicy. But others might argue that bananas are the best because they're creamy and easy to eat.

Nutritionally speaking, blueberries are often praised for their antioxidants, which are good for health. Avocados are high in healthy fats and nutrients too. Oranges are packed with vitamin C. Then there's the consideration of where fruits come from—some might prefer locally grown fruits bec